In [1]:
import pandas as pd 
import numpy as np
import seaborn as sns
import sklearn as sk
import matplotlib.pyplot as plt
import json
from pandas import json_normalize
from dateutil import parser
import nltk
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\Usuario\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [2]:
"This is to load the games database onto a pandas dataframe, which allows us to manage the data more easily"
games=pd.read_json("output_steam_games.json",lines=True)

In [3]:
"Here, we  will check the head of the games database to see if the information is correct"
games.info()
gameswna=games.dropna(axis='index',how='all')
gameswna.reset_index(inplace=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120445 entries, 0 to 120444
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   publisher     24083 non-null  object 
 1   genres        28852 non-null  object 
 2   app_name      32133 non-null  object 
 3   title         30085 non-null  object 
 4   url           32135 non-null  object 
 5   release_date  30068 non-null  object 
 6   tags          31972 non-null  object 
 7   reviews_url   32133 non-null  object 
 8   specs         31465 non-null  object 
 9   price         30758 non-null  object 
 10  early_access  32135 non-null  float64
 11  id            32133 non-null  float64
 12  developer     28836 non-null  object 
dtypes: float64(2), object(11)
memory usage: 11.9+ MB


In [4]:
gameswna.dropna(axis=0,how='any',inplace=True)
gameswna.drop(['publisher','index','app_name','url','tags','reviews_url','specs','price','early_access'],axis=1,inplace=True)

C:\Users\Usuario\AppData\Local\Temp\ipykernel_20752\2096993274.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gameswna.dropna(axis=0,how='any',inplace=True)
C:\Users\Usuario\AppData\Local\Temp\ipykernel_20752\2096993274.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gameswna.drop(['publisher','index','app_name','url','tags','reviews_url','specs','price','early_access'],axis=1,inplace=True)


In [5]:
gameswna.head()

,genres,title,release_date,id,developer
0,"[Action, Casual, Indie, Simulation, Strategy]",Lost Summoner Kitty,2018-01-04,761140.0,Kotoshiro
1,"[Free to Play, Indie, RPG, Strategy]",Ironbound,2018-01-04,643980.0,Secret Level SRL
2,"[Casual, Free to Play, Indie, Simulation, Sports]",Real Pool 3D - Poolians,2017-07-24,670290.0,Poolians.com
3,"[Action, Adventure, Casual]",弹炸人2222,2017-12-07,767400.0,彼岸领域
5,"[Action, Adventure, Simulation]",Battle Royale Trainer,2018-01-04,772540.0,Trickjump Games Ltd


In [6]:
gameswna['genres']=gameswna['genres'].apply(lambda x:', '.join(x))
gameswna['release_year'] = gameswna['release_date'].str.extract(r'(\d{4})', expand=False)

C:\Users\Usuario\AppData\Local\Temp\ipykernel_20752\3536262869.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gameswna['genres']=gameswna['genres'].apply(lambda x:', '.join(x))
C:\Users\Usuario\AppData\Local\Temp\ipykernel_20752\3536262869.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gameswna['release_year'] = gameswna['release_date'].str.extract(r'(\d{4})', expand=False)


In [7]:
gameswna_dummies = gameswna['genres'].str.get_dummies(', ')
gameswna = pd.concat([gameswna, gameswna_dummies], axis=1)
gameswna.drop(['genres','release_date'],axis=1,inplace=True)

In [9]:
gameswna.to_csv('clean_output_steam_games.csv',index=False)

In [10]:
"This step is done to load the reviews database onto a pandas dataframe, which allows us to manage the data in a more efficient way"
with open("australian_user_reviews.json",'r', encoding ='utf-8') as rw:
    data=rw.readlines()
db=[eval(line.strip()) for line in data]
reviews=pd.DataFrame(db)

In [11]:
"Here, we  will check the head of the reviews database to see if the information is correct"
reviews.info()
reviewswna=reviews.dropna(axis='index',how='all')
reviewswna.reset_index(inplace=True)
reviewswna.drop(['user_url','index'],axis=1,inplace=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25799 entries, 0 to 25798
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   user_id   25799 non-null  object
 1   user_url  25799 non-null  object
 2   reviews   25799 non-null  object
dtypes: object(3)
memory usage: 604.8+ KB


In [12]:
reviewswna.head()

,user_id,reviews
0,76561197970982479,"[{'funny': '', 'posted': 'Posted November 5, 2..."
1,js41637,"[{'funny': '', 'posted': 'Posted June 24, 2014..."
2,evcentric,"[{'funny': '', 'posted': 'Posted February 3.',..."
3,doctr,"[{'funny': '', 'posted': 'Posted October 14, 2..."
4,maplemage,"[{'funny': '3 people found this review funny',..."


In [13]:
# Since the reviews column database has more than one review, we use the explode function in order to separate every review done by the user and we store it into a new table.
df_exploded = reviewswna.explode('reviews')
# Afterwards, we reset the index in order to match the amount of columns, which is 59333.
df_exploded.reset_index(inplace=True)
# By using the drop index, we make sure to reset every single index in order for it to show us the entirety of the 59333 indexes by getting rid of the old indexes.
df_exploded.drop('index',axis=1,inplace=True)
# On the other hand, by using the json_normalize function, we make sure to separate the fields on the reviews column, this for us to get rid of the information we don't need or won't be using.
df_exploded1=json_normalize(reviewswna['reviews'].explode())

In [14]:
# Here, we check the amount of rows of the new table we created.
#df_exploded #59333 rows
# Likewise
#df_exploded1 #59333 rows
# With this line we make sure to merge the split columns of the review section and the split rows of the user_id so there are no null users when merging.
reviewswna1=pd.concat([df_exploded,df_exploded1],axis=1)
# We proceed to get rid of the columns we wont be using towards the objective of the project and also remove any Nas that were created in the process, and also reset the indexes.
reviewswna1.drop(['reviews','helpful','last_edited','funny'], axis=1, inplace=True)
reviewswna1.dropna(inplace=True)
reviewswna1.reset_index(inplace=True)
reviewswna1.drop('index',axis=1,inplace=True)

In [15]:
# By using dateutil.parser, we create a function in order to extract the year from the posted column
def extract_year(date_string):
    try:
        # We try to analyze the date with various formats
        parsed_date = parser.parse(date_string)
        return parsed_date.year
    except ValueError:
        # If this doesn't work, we try another format.
        try:
            parsed_date = parser.parse(date_string, fuzzy=True)
            return parsed_date.year
        except ValueError:
            # If it fails again, it just returns NaT(not a time)
            return pd.NaT

In [17]:
# By applying the function we just created, we add a new column to our dataframe called year
reviewswna1['year']=reviewswna1['posted'].apply(extract_year)
# By using the unique function, we evaluate outliers to see why are they being created, in this case, we identify that there's an outlier with the year 2024, when we check the database,
# we discover that the rows that have the 2024 didn't have a year on the posted column, so we proceed to delete the rows that contain the 2024 because that number was assigned by default since the field didn't have a year.
reviewswna1['year'].unique()

array([2011, 2014, 2013, 2024, 2015, 2012, 2010], dtype=int64)

In [22]:
reviewswna2f

,user_id,item_id,recommend,review,year
0,76561197970982479,1250,True,Simple yet with great replayability. In my opi...,2011
1,76561197970982479,22200,True,It's unique and worth a playthrough.,2011
2,76561197970982479,43110,True,Great atmosphere. The gunplay can be a bit chu...,2011
3,js41637,251610,True,I know what you think when you see this title ...,2014
4,js41637,227300,True,For a simple (it's actually not all that simpl...,2013
...,...,...,...,...,...
49181,wayfeng,730,True,its FUNNNNNNNN,2015
49182,76561198251004808,253980,True,Awesome fantasy game if you don't mind the gra...,2015
49183,72947282842,730,True,Prettyy Mad Game,2015
49184,ApxLGhost,730,True,AMAZING GAME 10/10,2015


In [20]:
# We apply a filter to get rid of the 2024 values, afterwards, we proceed to reset the indexes and finally, finish cleaning the table.
reviewswna2f=reviewswna1[reviewswna1['year']!=2024]
reviewswna2f.reset_index(inplace=True)
reviewswna2f.drop('index',axis=1,inplace=True)
reviewswna2f.drop('posted',axis=1,inplace=True)
reviewswna2f['item_id']=reviewswna2f['item_id'].astype(int)
reviewswna2f['year']=reviewswna2f['year'].astype(int)

C:\Users\Usuario\AppData\Local\Temp\ipykernel_20752\3581548502.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reviewswna2f.drop('index',axis=1,inplace=True)
C:\Users\Usuario\AppData\Local\Temp\ipykernel_20752\3581548502.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reviewswna2f.drop('posted',axis=1,inplace=True)
C:\Users\Usuario\AppData\Local\Temp\ipykernel_20752\3581548502.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/

In [23]:
# Initialize the sentiment analyzer
sia = SentimentIntensityAnalyzer()

# Function to assign values according to the scale
def get_sentiment_score(text):
    if pd.isnull(text) or text == '':
        return 1  # Return neutral if it is empty or NaN
    elif isinstance(text, str):
        sentiment = sia.polarity_scores(text)
        compound_score = sentiment['compound']
        if compound_score >= -0.05:
            return 2  # Good score
        elif compound_score <= -0.05:
            return 0  # Bad score
        else:
            return 1
    else:
        return 1  # Return neutral for non-string values

In [25]:
#reviewswna2f.drop('sentiment_analysis',axis=1,inplace=True)
reviewswna2f['sentiment_analysis']=reviewswna2f['review'].apply(get_sentiment_score)

C:\Users\Usuario\AppData\Local\Temp\ipykernel_20752\2489924637.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reviewswna2f['sentiment_analysis']=reviewswna2f['review'].apply(get_sentiment_score)


In [26]:
# We check our dataframe to see if it does have the information we need before saving it to a csv
reviewswna2f
# We save our dataframe into a csv database.
reviewswna2f.to_csv('clean_output_user_reviews.csv',index=False)

In [27]:
"This step is done to load the items database onto a pandas dataframe, which allows us to manage the data in a more efficient way"
with open("australian_users_items.json",'r',encoding='utf-8') as it:
    data=it.readlines()
db1=[eval(line.strip()) for line in data]
items=pd.DataFrame(db1)

In [28]:
"Here, we  will check the head of the items database to see if the information is correct"
items.info()
itemswna=items.dropna(axis='index',how='all')
itemswna.reset_index(inplace=True)
#Here we are deleting from our dataframe the columns that we don't need, in this case, we reset the indexes if there's a change on them after deleting the duplicates and Na's
itemswna.drop(['index','steam_id','user_url','items_count'],axis=1,inplace=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 88310 entries, 0 to 88309
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   user_id      88310 non-null  object
 1   items_count  88310 non-null  int64 
 2   steam_id     88310 non-null  object
 3   user_url     88310 non-null  object
 4   items        88310 non-null  object
dtypes: int64(1), object(4)
memory usage: 3.4+ MB


In [29]:
itemswna.head()

,user_id,items
0,76561197970982479,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
1,js41637,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
2,evcentric,"[{'item_id': '1200', 'item_name': 'Red Orchest..."
3,Riot-Punch,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
4,doctr,"[{'item_id': '300', 'item_name': 'Day of Defea..."


In [30]:
itemsf=itemswna.explode('items')
itemsf.reset_index(inplace=True)
itemsf.drop('index',axis=1,inplace=True)
itemsf2=json_normalize(itemswna['items'].explode())
itemswnaf=pd.concat([itemsf,itemsf2],axis=1)
itemswnaf.drop(['items','playtime_2weeks'],axis=1,inplace=True)
itemswnaf.dropna(axis=0,how='any',inplace=True)
itemswnaf.reset_index(inplace=True)
itemswnaf.drop('index',axis=1,inplace=True)

In [31]:
#itemsf #5170015 rows
#itemsf2 #5170015 rows
itemswnaf#5153209 rows
itemswnaf.to_csv('clean_output_user_items.csv',index=False)

In [32]:
# Since we already have the information stored as CSV format, we will proceed by reading them and saving them into pandas dataframes.
items=pd.read_csv('clean_output_user_items.csv')
reviews=pd.read_csv('clean_output_user_reviews.csv')
games=pd.read_csv('clean_output_steam_games.csv')

In [56]:
# We will check the first 5 rows of each database in order to see the column that will be used to merge both databases.
games.head()
items.head()

,user_id,item_id,item_name,playtime_forever
0,76561197970982479,10,Counter-Strike,6.0
1,76561197970982479,20,Team Fortress Classic,0.0
2,76561197970982479,30,Day of Defeat,7.0
3,76561197970982479,40,Deathmatch Classic,0.0
4,76561197970982479,50,Half-Life: Opposing Force,0.0


In [42]:
# With this code lines what we're doing is merging the items database and the games database, in order to have the information we need in one database.
# We also clean any na's that may appear, reset the indexes and drop the columns that we no longer need or are duplicated, finally we store the new merged database into a CSV file.
maindf=pd.merge(items,games,how='outer',left_on='item_id',right_on='id')
maindf1=maindf.dropna(axis=0,how='any')
maindf1.reset_index(inplace=True)
maindf1.drop(['item_id','item_name','index'],axis=1,inplace=True)
maindf1.to_csv('maindb1.csv',index=False)

C:\Users\Usuario\AppData\Local\Temp\ipykernel_20752\1795636734.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  maindf1.drop(['item_id','item_name','index'],axis=1,inplace=True)


In [2]:
alldb=pd.read_csv('maindb1.csv')
alldbgames=alldb.drop_duplicates(subset='title')
alldbgames.drop(['release_year','user_id','playtime_forever','Action','Adventure','Audio Production','Early Access','Photo Editing','RPG','Racing','Simulation','Software Training','Sports','Web Publishing','Utilities','Strategy','Video Production','Casual','Animation &amp; Modeling','Design &amp; Illustration','Education','Free to Play','Indie','Design &amp; Illustration','Massively Multiplayer'],axis=1,inplace=True)
alldbgames.reset_index(inplace=True)
alldbgames.drop('index',axis=True,inplace=True)
alldbgames.to_csv('steam_games_devel.csv')

C:\Users\Usuario\AppData\Local\Temp\ipykernel_7912\3290017874.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  alldbgames.drop(['release_year','user_id','playtime_forever','Action','Adventure','Audio Production','Early Access','Photo Editing','RPG','Racing','Simulation','Software Training','Sports','Web Publishing','Utilities','Strategy','Video Production','Casual','Animation &amp; Modeling','Design &amp; Illustration','Education','Free to Play','Indie','Design &amp; Illustration','Massively Multiplayer'],axis=1,inplace=True)
C:\Users\Usuario\AppData\Local\Temp\ipykernel_7912\3290017874.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-

In [3]:
maindb1=alldb.head(200000)
maindb1.to_csv('maindb.csv')

In [46]:
maindb_gamenames=maindb.drop_duplicates(subset='title')
merged_df1=pd.merge(maindb_gamenames,reviews,right_on='item_id',left_on='id')
merged_df1.drop(['user_id_y','item_id','release_year','developer','playtime_forever','year'],axis=1,inplace=True)
merged_df1.to_csv('sample.csv')

In [47]:
merged_df1

,user_id_x,title,id,Action,Adventure,Animation &amp; Modeling,Audio Production,Casual,Design &amp; Illustration,Early Access,...,Simulation,Software Training,Sports,Strategy,Utilities,Video Production,Web Publishing,recommend,review,sentiment_analysis
0,76561197970982479,Counter-Strike,10.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,True,Cool game,2
1,76561197970982479,Counter-Strike,10.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,True,Wallbang simulator,2
2,76561197970982479,Counter-Strike,10.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,True,this game is the 1# online action game is awes...,2
3,76561197970982479,Counter-Strike,10.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,True,THE BEST FPS GAME!!!!!,2
4,76561197970982479,Counter-Strike,10.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,True,One of the best childhood games i have ever pl...,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36569,PinkamenaDianePyro,Gardens Inc. – From Rakes to Riches,279420.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,True,Pros - Doesn't require a super computer to run...,2
36570,BestKingpin,World of Subways 1 – The Path,306370.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,False,"pile of garbage, DON'T BUY.",2
36571,76561198026335213,Shrooms,361330.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,False,loading simulator 2015 (multiplayer doesnt work.),2
36572,76561197970580235,Transcendence,364510.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,True,i've played this game ever since i was a littl...,2
